In [44]:
import pandas as pd
import pickle
from search_route import search_route

with open('p5r_data.pkl', 'rb') as file:
    data = pickle.load(file)

df_persona = pd.read_csv('p5r_人格面具.csv', index_col=0)
df_compose = pd.read_csv('p5r_合成总表.csv')
persona_level = dict(zip(df_persona['中文名'].tolist(), df_persona['等级'].tolist()))
df_compose['Level'] = df_compose['Result'].map(persona_level)

In [ ]:
# 查询反向合成表
df_compose.loc[df_compose['Result'] == '八岐大蛇']

,价格,种族,Lv,Name,种族.1,Lv.1,Name.1,Result
6,6084.0,魔术师,8,猫妖精,魔术师,11.0,杰克霜精,杰克灯笼


In [67]:
# 查询技能
name = '八岐大蛇'
data[name][-2].iloc[4:, 1:6]

,1,2,3,4,5
4,技能,消耗,解锁等级,解锁等级,描述
5,一枪毙命,17% HP,自带,自带,对1名敌人造成高暴击率的枪击属性特大伤害
6,玛哈布芙达因,22 SP,自带,自带,对所有敌人造成冰冻属性的大量伤害，概率使敌人冻结
7,冻结机率UP,Auto,自带,自带,冻结的附加率自动上升
8,冰冻吸收,Auto,66,66,冰冻属性伤害吸收
9,高级冰冻强化,Auto,67,67,冰冻属性攻击提升50%
10,钻石星尘,48 SP,69,69,对1名敌人造成特大冰冻属性伤害并有机率附带冰冻效果


In [68]:
# 查询任意种族，等级面具
df_persona.loc[df_persona['中文名'].str.contains('八岐大蛇')]

,中文名,等级,英文名,日文名,塔罗牌,物,枪,火,冰,电,风,念,核,祝,咒
174,八岐大蛇,64,Yamata-no-Orochi,ヤマタノオロチ,审判,耐,-,-,吸,-,耐,-,-,-,耐


In [59]:
# 查询反向合成表
df_compose.loc[df_compose['Result'].str.contains('妖精女王')]

,价格,种族,Lv,Name,种族.1,Lv.1,Name.1,Result,Level
7971,20634.0,命运之轮,30,阿里亚德涅,女教皇,1645.0,伊西丝,妖精女王,43
7972,20634.0,星星,30,猫将军,愚者,1646.0,俄耳普斯,妖精女王,43
7973,20724.0,月,25,辉夜·贼神,刚毅,1647.0,增长天,妖精女王,43
7974,21210.0,太阳,34,闪电鸟,节制,1648.0,持国天,妖精女王,43
7975,21477.0,月,33,天蛾人,刚毅,1649.0,罗刹,妖精女王,43
...,...,...,...,...,...,...,...,...,...
8249,70020.0,魔术师,5,猫妖精,魔术师,1923.0,斯尔特,妖精女王,43
8250,71136.0,魔术师,11,杰克霜精,魔术师,1924.0,斯尔特,妖精女王,43
8251,71181.0,星星,84,哈斯塔,愚者,1925.0,亚森,妖精女王,43
8252,74394.0,魔术师,2,杰克灯笼,魔术师,1926.0,经津主,妖精女王,43


In [64]:
# 查询一步能合成的东西
name = '莉莉姆'
level = "40 - 45"

level_range = level.split(' - ')
name_cond = (df_compose['Name'].str.contains(name)) | (df_compose['Name.1'].str.contains(name))
level_cond = (df_compose['Level'] > int(level_range[0])) & (df_compose['Level'] < int(level_range[1]))
df_compose.loc[name_cond & level_cond]
# df_compose.loc[name_cond & level_cond, ['Result', 'Level']].drop_duplicates('Result')

,价格,种族,Lv,Name,种族.1,Lv.1,Name.1,Result,Level
6746,27834.0,恶魔,32,莉莉姆,皇帝,9082.0,图特,能天使,41
7064,26490.0,审判,34,阿努比斯,恶魔,7144.0,莉莉姆,奇稻田姬,42
7259,21333.0,恶魔,32,莉莉姆,教皇,12912.0,安祖,百臂巨人,42
7315,30738.0,恶魔,32,莉莉姆,教皇,12968.0,大僧正,百臂巨人,42
7452,34773.0,恶魔,32,莉莉姆,节制,593.0,白虎,欧赛,42
7570,29985.0,太阳,39,米特拉斯,恶魔,10076.0,莉莉姆,鸠盘荼,42
7582,33105.0,太阳,43,虹蛇,恶魔,10088.0,莉莉姆,鸠盘荼,42
7610,36513.0,太阳,47,荷鲁斯,恶魔,10116.0,莉莉姆,鸠盘荼,42
7724,30738.0,恶魔,32,莉莉姆,女教皇,18818.0,菊理媛,基力梅卡拉,43
7734,32298.0,恶魔,32,莉莉姆,恋人,18828.0,奇稻田姬,基力梅卡拉,43


In [43]:
df_compose['Lv.1'].max()

23003.0

In [15]:
# 查询成本最低合成路径

start = '阿努比斯'
end = '大僧正'
head_num = 10
max_depth = 3

routes = search_route(start, end, max_depth=max_depth)
for idx, route in enumerate(routes):
    if idx < head_num:
        print(route[0], '=', route[1])
    else:
        break

阿努比斯 + 杰克灯笼 -> 鬼 + 鞍马天狗 -> 大僧正 = 34539.0
阿努比斯 + 猫妖精 -> 鬼 + 鞍马天狗 -> 大僧正 = 34854.0
阿努比斯 + 亚森 -> 闪电鸟 + 猫妖 -> 大僧正 = 35226.0
阿努比斯 + 奇魂 -> 安德拉斯 + 睡魔 -> 大僧正 = 35250.0
阿努比斯 + 杰克霜精 -> 罗刹 + 毕舍遮 -> 大僧正 = 36003.0
阿努比斯 + 背负怪 -> 闪电鸟 + 猫妖 -> 大僧正 = 36087.0
阿努比斯 + 埃力格 -> 伊西丝 + 水鬼 -> 大僧正 = 36372.0
阿努比斯 + 亚森 -> 闪电鸟 + 欧特鲁斯 -> 大僧正 = 36762.0
阿努比斯 + 皮克希 -> 欧特鲁斯 + 闪电鸟 -> 大僧正 = 36831.0
阿努比斯 + 埃力格 -> 伊西丝 + 金鬼 -> 大僧正 = 36855.0
